In [4]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import json
import spotipy

In [7]:
# Example from spotipy website to check that it works
"""
Credentials can be set on your local computer by following this link: https://www.youtube.com/watch?v=kaBVN8uP358&t=0s
"""
from spotipy.oauth2 import SpotifyClientCredentials

birdy_uri = 'spotify:artist:2WX2uTcsvV5OnS0inACecP'
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

results = spotify.artist_albums(birdy_uri, album_type='album')
albums = results['items']
while results['next']:
    results = spotify.next(results)
    albums.extend(results['items'])

for album in albums:
    print(album['name'])

Young Heart
Beautiful Lies
Beautiful Lies
Beautiful Lies (Deluxe)
Beautiful Lies (Deluxe)
Fire Within
Fire Within
Fire Within (Deluxe)
Fire Within (Deluxe)
Fire Within (Deluxe)
Live in London
Birdy
Birdy
Birdy
Birdy
Birdy (Deluxe Version)


In [8]:
# path to playlist file
root_path = !pwd
root_path = str(root_path[0])

In [9]:
# filename 
filepath = f"{root_path}/playlist_data/sampledata/"
filename = 'mpd.slice.0-999.json'

In [10]:
# path + filename
fpath_name = f"{filepath}{filename}"
# print(fpath_name)

In [11]:
# Open file of playlist and obtain fields of json file 
with open(fpath_name) as data_file:    
    data = json.load(data_file)  

In [12]:
# Explore fields in json file
print("File keys: ", list(data.keys()))
print("info: ",data[list(data.keys())[0]])
#print(f"{list(data.keys())[1]} : ",data[list(data.keys())[1]])
print("playlists: ",list(data['playlists'][0].keys()))
print("tracks: ", list(data['playlists'][0]['tracks'][0].keys()))

File keys:  ['info', 'playlists']
info:  {'generated_on': '2017-12-03 08:41:42.057563', 'slice': '0-999', 'version': 'v1'}
playlists:  ['name', 'collaborative', 'pid', 'modified_at', 'num_tracks', 'num_albums', 'num_followers', 'tracks', 'num_edits', 'duration_ms', 'num_artists']
tracks:  ['pos', 'artist_name', 'track_uri', 'artist_uri', 'track_name', 'album_uri', 'duration_ms', 'album_name']


In [13]:
# Extract track features
num_keys_old = 0
for index, playlist in enumerate(data['playlists']):
    num_keys =  len(playlist.keys())
    if index == 0:
        num_key_old = num_keys
        continue
    if num_keys > num_key_old:
        keys = playlist.keys()
    num_key_old = num_keys

In [14]:
# Construct keys of playlist
keys = list(keys)
keys.remove('tracks')
print(" Playlist keys: ", keys)

 Playlist keys:  ['name', 'collaborative', 'pid', 'modified_at', 'num_tracks', 'num_albums', 'num_followers', 'num_edits', 'duration_ms', 'num_artists', 'description']


In [15]:
# Create dataframe with track and playlist info
music_df = pd.json_normalize(data['playlists'],  meta = keys, meta_prefix = 'playlist_', errors='ignore', record_path=['tracks'], record_prefix = 'track_')
music_df.head()

,track_pos,track_artist_name,track_track_uri,track_artist_uri,track_track_name,track_album_uri,track_duration_ms,track_album_name,playlist_name,playlist_collaborative,playlist_pid,playlist_modified_at,playlist_num_tracks,playlist_num_albums,playlist_num_followers,playlist_num_edits,playlist_duration_ms,playlist_num_artists,playlist_description
0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
1,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
2,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
3,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
4,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN


In [16]:
# Check column values 
music_df[music_df['playlist_pid'] == 0].columns

Index(['track_pos', 'track_artist_name', 'track_track_uri', 'track_artist_uri',
       'track_track_name', 'track_album_uri', 'track_duration_ms',
       'track_album_name', 'playlist_name', 'playlist_collaborative',
       'playlist_pid', 'playlist_modified_at', 'playlist_num_tracks',
       'playlist_num_albums', 'playlist_num_followers', 'playlist_num_edits',
       'playlist_duration_ms', 'playlist_num_artists', 'playlist_description'],
      dtype='object')

In [17]:
# Select indexes of playlists that have a description
playlists_id_with_description = music_df[~music_df['playlist_description'].isna()]['playlist_pid'].unique()
print(playlists_id_with_description)

[94 102 320 329 339 353 354 370 475 491 522 526 548 626 741 767 778 818
 871 925]


In [18]:
# Working only with playlists that contain a "description"
subset_df = music_df[music_df['playlist_pid'].isin(playlists_id_with_description)]
print(subset_df['playlist_pid'].unique())

[94 102 320 329 339 353 354 370 475 491 522 526 548 626 741 767 778 818
 871 925]


In [19]:
subset_df.head()

,track_pos,track_artist_name,track_track_uri,track_artist_uri,track_track_name,track_album_uri,track_duration_ms,track_album_name,playlist_name,playlist_collaborative,playlist_pid,playlist_modified_at,playlist_num_tracks,playlist_num_albums,playlist_num_followers,playlist_num_edits,playlist_duration_ms,playlist_num_artists,playlist_description
5431,0,James Bay,spotify:track:13HVjjWUZFaWilh2QUJKsP,spotify:artist:4EzkuveR9pLvDVFNx6foYD,Let It Go,spotify:album:5BxvswQSGWrBbVCdx6mFGO,260533,Chaos And The Calm,relax,false,94,1495324800,124,112,1,35,27578241,97,chilllll out
5432,1,Passenger,spotify:track:4jpwHDjemKbmNy0pyWDHpr,spotify:artist:0gadJ2b9A4SKsB1RFkBb66,All the Little Lights,spotify:album:0kjXSwSLVipUi4lSEeF0yl,233973,All The Little Lights,relax,false,94,1495324800,124,112,1,35,27578241,97,chilllll out
5433,2,The Script,spotify:track:1a1i2IwXnUN0PjllVEhA4B,spotify:artist:3AQRLZ9PuTAozP28Skbq8V,Breakeven,spotify:album:5gCo4z8XZ0T16nJmnw9wnG,261426,The Script,relax,false,94,1495324800,124,112,1,35,27578241,97,chilllll out
5434,3,Lukas Graham,spotify:track:7129iqBafaphfc3WPCGC0L,spotify:artist:25u4wHJWxCA9vO0CzxAbK7,7 Years,spotify:album:6NQ2RNksdSKdtnlFOLJo24,237300,Lukas Graham (Blue Album),relax,false,94,1495324800,124,112,1,35,27578241,97,chilllll out
5435,4,Jason Castro,spotify:track:6hsP7b83RjEY51ENfo1ZOc,spotify:artist:6lv3DKGOLy7HkRUmt6DiEd,Hallelujah,spotify:album:0xtQMHjzZDrvwe8PbBNAgc,233240,Jason Castro,relax,false,94,1495324800,124,112,1,35,27578241,97,chilllll out


In [22]:
# Pull metadata from a single track 
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

track_uri = subset_df.iloc[0]['track_track_uri']


results = spotify.audio_features(tracks=track_uri)
print(results)
# albums = results['items']
# while results['next']:
#     results = spotify.next(results)
#     albums.extend(results['items'])

# for album in albums:
#     print(album['name'])

[{'danceability': 0.546, 'energy': 0.311, 'key': 1, 'loudness': -10.396, 'mode': 1, 'speechiness': 0.0288, 'acousticness': 0.818, 'instrumentalness': 1.65e-05, 'liveness': 0.107, 'valence': 0.246, 'tempo': 147.464, 'type': 'audio_features', 'id': '13HVjjWUZFaWilh2QUJKsP', 'uri': 'spotify:track:13HVjjWUZFaWilh2QUJKsP', 'track_href': 'https://api.spotify.com/v1/tracks/13HVjjWUZFaWilh2QUJKsP', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/13HVjjWUZFaWilh2QUJKsP', 'duration_ms': 260533, 'time_signature': 4}]


In [ ]:
#Ntracks = 20 
# print(music_df[music_df['playlist_num_tracks'] == Ntracks]['track_artist_name'].unique())
#subset_df = music_df[music_df['playlist_num_tracks'] == Ntracks]
print('Number of records: ', len(subset_df))
print('Number of unique artist, song pairs: ', len(subset_df[['track_artist_name','track_track_name']].apply(lambda x: ',.'.join(x),axis=1).unique()))
print('Number of unique artists: ', len(subset_df['track_artist_name'].unique()))

In [ ]:
# Work only with important features
important_features = ['track_artist_name', 'track_track_name', 'playlist_name', 'playlist_duration_ms', 'playlist_description']

In [ ]:
artist_count_df = subset_df[['playlist_name','track_artist_name']].groupby(['playlist_name'], sort=False).value_counts(sort=False).reset_index(name="artist_count")

In [ ]:
artist_count_df

In [ ]:
subset_df[['playlist_name', 'track_artist_name']]

In [ ]:
subset_df[['playlist_name','track_artist_name']].groupby(['playlist_name'], sort = False).value_counts(sort=False).reset_index(name="artist_count").info()

In [ ]:
music_df = pd.merge(subset_df, artist_count_df,  how='left', on = ['playlist_name','track_artist_name'])

In [ ]:
music_df

In [ ]:
# subset_df.join(.sum(), on='playlist_name', rsuffix='_count')
subset_df.groupby(['playlist_name','track_artist_name']).size().unstack(fill_value=0)

In [ ]:
music_df[['track_artist_name','track_track_name']].apply(lambda x: ',.'.join(x),axis=1)

In [ ]:
important_features.append("artist_count")

In [ ]:
music_df[important_features]

In [ ]:
music_df['artist_fraction'] =  music_df['artist_count'] / music_df['playlist_num_tracks'] 
music_df

In [ ]:
subset_df.groupby(['playlist_name','track_artist_name']).size().unstack(fill_value=0)

In [ ]:
music_df[music_df['playlist_num_tracks'] == Ntracks][['track_artist_name','track_track_name']].apply(lambda x: ',.'.join(x),axis=1).value_counts() 

In [ ]:
vc = music_df[music_df['playlist_num_tracks'] == Ntracks][['track_artist_name','track_track_name']].apply(lambda x: ',.'.join(x), axis=1).value_counts()
vc[vc > 1].sum() - len(vc[vc > 1])

In [ ]:
# for i in df[['track_artist_name','track_track_name']].apply(lambda x: ',.'.join(x),axis=1).unique():
# #     print(i.split(',.'))
#     if len(i.split(',.')) == 2:
#         print(i)

In [ ]:
np.unique(music_df[music_df['playlist_num_tracks'] == Ntracks]['track_track_name'])

In [ ]:
important_features = ['track_artist_name', 'track_track_name', 'playlist_name', 'playlist_num_tracks', 'playlist_num_albums', 'playlist_duration_ms', 'playlist_num_artists']

In [ ]:
music_df[music_df['playlist_num_tracks'] == 20][['track_artist_name', 'track_track_name', 'playlist_name', 'playlist_num_tracks', 'playlist_num_albums', 'playlist_duration_ms', 'playlist_num_artists', 'playlist_description']]

In [ ]:
music_df[music_df['playlist_num_tracks'] == Ntracks][important_features].groupby('playlist_name')['track_artist_name'].value_counts()